In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install numpy requests nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Load Dataset

In [ ]:
import random
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

from nlpaug.util import Action

In [ ]:
SEED = 1234

In [ ]:
alpha_common_error = 0.10
alpha_common_error_char = 0.05
aug1_OCR = nac.OcrAug(aug_word_p=alpha_common_error)
aug2_Rins = nac.RandomCharAug(action="insert", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug3_Rsub = nac.RandomCharAug(action="substitute", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug4_Rswa = nac.RandomCharAug(action="swap", aug_word_p=alpha_common_error,aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char) #
aug5_Rdel = nac.RandomCharAug(action="delete", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug6_Kb = nac.KeyboardAug(aug_word_p=alpha_common_error)
aug7_Split = naw.SplitAug(aug_p=alpha_common_error)

In [ ]:
random.sample(range(0, 7), 2 - 1)

[6]

In [ ]:
def text2augment(text, m):
    output = [text, ]

    temp = random.sample(range(0, 7), m - 1)

    if 0 in temp:
        output.append(*aug1_OCR.augment(text))
    if 1 in temp:
        output.append(*aug2_Rins.augment(text))
    if 2 in temp:
        output.append(*aug3_Rsub.augment(text))
    if 3 in temp:
        output.append(*aug4_Rswa.augment(text))
    if 4 in temp:
        output.append(*aug5_Rdel.augment(text))
    if 5 in temp:
        output.append(*aug6_Kb.augment(text))
    if 6 in temp:
        output.append(*aug7_Split.augment(text))

    return output

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def to_df(x, y):
    d = {"text": x, "label": y}
    return pd.DataFrame(d)

def split_3(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y,random_state=SEED)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y,random_state=SEED)
    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
from itertools import chain

In [ ]:
def aug_replicate(y_labels):
    #return list(chain(* [[y]*(2 if y == 0 else 4) for y in y_labels] ))
    #return list(chain(* [[y]*(1 if y == 0 else 3) for y in y_labels] ))
    #return list(chain(* [[y]*(1 if y == 0 else 2) for y in y_labels] ))
    return list(chain(* [[y]*(2 if y == 0 else 3) for y in y_labels] ))


def aug_text(x_text, y_labels):
    #x_text = [ text2augment(x, 2 if y == 0 else 4) for x, y in zip(x_text, y_labels)]
    #x_text = [ text2augment(x, 1 if y == 0 else 3) for x, y in zip(x_text, y_labels)]
    #x_text = [ text2augment(x, 1 if y == 0 else 2) for x, y in zip(x_text, y_labels)]
    x_text = [ text2augment(x, 2 if y == 0 else 3) for x, y in zip(x_text, y_labels)]
    return pd.Series(list(chain(*x_text)), index=None)

def split_3_aug(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y,random_state=SEED)
    # augment
    # x_test = aug_text(x_test, y_test)
    # y_test = aug_replicate(y_test)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y,random_state=SEED)
    # augment
    x_train = aug_text(x_train, y_train)
    y_train = aug_replicate(y_train)
    x_valid = aug_text(x_valid, y_valid)
    y_valid = aug_replicate(y_valid)

    print(x_valid.shape)
    print("DONE")
    print(len(y_valid))

    print(x_train.shape)
    print("DONE")
    print(len(y_train))

    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
import numpy as np
import pandas as pd

tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/hsd_merge_cleaned_lowered"
data = pd.read_csv(f"{tname_data}.csv")

train, valid, test = split_3_aug(data)

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

(6216,)
DONE
6216
(24856,)
DONE
24856


In [ ]:
# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_train.csv"
# train.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_valid.csv"
# valid.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_test.csv"
# test.to_csv(temp,  sep=",", index=False)

In [ ]:
data[data["label"] == 1]

,text,label
0,pillaron al presunto es de nacionalidad colomb...,1
1,cuando se den cuenta que eran homicidas o narc...,1
2,jajaajja ahí estuviste bien chouza se nota tu ...,1
3,acá podemos ver a un alemán de cabello rubio o...,1
4,chile murieron 8 compatriotas por culpa de un ...,1
...,...,...
17778,marinovic hay alguna mujer del movimiento femi...,1
17782,leí tu descripción ahora entiendo todo el daño...,1
17816,🤮🤮🤮🤮una 💩 de debate solo orrego se presta para...,1
17822,los delincuentes de la primers linea queman ig...,1


In [ ]:
train[train["label"] == 1]

,text,label
8,para pagar pensiones 4 detenidos por introduci...,1
9,para pagtr pensiones 4 detenidos por intrfduci...,1
10,para pagar pensiones 4 detenidos por introduci...,1
21,que se vaya ya el puto negro de mierda,1
22,que se vaya ya el puto negro de mieFda,1
...,...,...
24813,que me llamen racista pero esAa gentu^a deberí...,1
24814,que me llamen raista pero esa gentuza debería ...,1
24843,totalmente de acuerdo muchos de los emigrantes...,1
24844,totalmente de acuerdo muhcos de los emigrantes...,1


In [ ]:
valid[valid["label"] == 1]

,text,label
4,molecular pero si los de derecha son gente re ...,1
5,molecular pero si lu8 de derecha son gente re ...,1
6,molecular pe ro si los de derecha son gente re...,1
19,maricon como insulto un saludo,1
20,maricon cmo insulto un saludo,1
...,...,...
6201,el que tiene que parar el juego es el árbitro ...,1
6202,el que tiene que parra el juego es el árbitro ...,1
6205,un subnormal menos en twitter grande irene,1
6206,un subnormal enos en twitter grande irene,1


In [ ]:
test[test["label"] == 1]

,text,label
11451,que le jodan al mongolo de los huevos,1
5514,y digo yo ¿no sera la dimisión de o del entero...,1
15747,tienen para 140 000 € y pactar 100 plazas de j...,1
8352,hay un puto subnormal pitando en mi calle todo...,1
16499,tomasroncero estas jodido eh puto gordo maricon,1
...,...,...
12964,ok puto moro la proxima trae un contrato de in...,1
12393,aunque tu foto de perfil parece un marica el d...,1
5811,maigi si algo he aprendido es que el que tenga...,1
1706,tu prima es guarrilla,1


In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/original/dataset_9834_cleaned_lowered"
# data = pd.read_csv(f"{tname_data}.csv")
# # test = pd.read_csv(f"{tname_data}_test.csv")

# data = data.sample(frac=1).reset_index()
# data = data[["text", "label"]]

# train = data[: int(len(data) * 0.7)]
# valid = data[int(len(data) * 0.7) : int(len(data) * 0.9)]
# test = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']

In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "hsd_merged_AUG"

# data = pd.read_csv(f"{tname_data}_no_test.csv")
# test = pd.read_csv(f"{tname_data}_test.csv")


# train = data[: int(len(data) * 0.9)]
# valid = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']


## Set Cuda

In [ ]:
# import torch
# from torchtext import data
# from torchtext import datasets
# import random
# import numpy as np

# SEED = 1234

# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

# torch.cuda.is_available()

# Extract feature by using BETO

In [ ]:
#train_sentences

In [ ]:
import pandas as pd
from glob import glob

train_sentences = list(train['text'].values)
train_labels = list(train['label'].values)

valid_sentences = list(valid['text'].values)
valid_labels = list(valid['label'].values)

test_sentences = list(test['text'].values)
test_labels = list(test['label'].values)

In [ ]:
test_labelsdf = pd.DataFrame(test_labels)
test_labelsdf.to_csv('test_labels.csv',index=False)

Load tokenizer of BETO

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

In [ ]:
tokenizer.vocab_size

31002

In [ ]:
#choose max_length for beto model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(23, 740),
 (21, 734),
 (17, 727),
 (18, 704),
 (22, 698),
 (24, 694),
 (12, 686),
 (13, 685),
 (19, 680),
 (25, 668),
 (26, 660),
 (20, 649),
 (11, 638),
 (15, 628),
 (16, 626),
 (14, 612),
 (27, 592),
 (29, 569),
 (10, 558),
 (28, 543),
 (30, 523),
 (9, 519),
 (31, 489),
 (33, 477),
 (32, 470),
 (36, 416),
 (34, 412),
 (8, 387),
 (35, 368),
 (37, 337),
 (7, 334),
 (38, 306),
 (39, 268),
 (40, 266),
 (41, 247),
 (53, 245),
 (44, 235),
 (6, 235),
 (45, 234),
 (42, 220),
 (57, 213),
 (43, 210),
 (46, 210),
 (47, 209),
 (49, 208),
 (50, 204),
 (56, 200),
 (48, 197),
 (51, 193),
 (54, 191),
 (52, 190),
 (55, 184),
 (58, 167),
 (5, 165),
 (59, 165),
 (60, 156),
 (62, 152),
 (64, 144),
 (61, 142),
 (63, 141),
 (66, 127),
 (65, 114),
 (69, 105),
 (4, 97),
 (68, 87),
 (67, 84),
 (70, 71),
 (71, 71),
 (72, 64),
 (73, 46),
 (3, 38),
 (75, 32),
 (74, 29),
 (76, 26),
 (77, 25),
 (78, 19),
 (79, 17),
 (81, 15),
 (80, 14),
 (2, 13),
 (82, 7),
 (83, 7),
 (84, 5),
 (87, 5),
 (85, 4),
 (1, 4),
 (86, 

In [ ]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_train_labels = le.transform(train_labels)
encoded_valid_labels = le.transform(valid_labels)
encoded_test_labels = le.transform(test_labels)

In [ ]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=50,
                                             padding='max_length',
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_train_labels)
valid_sent_index,valid_input_ids,valid_attention_masks,valid_encoded_label_tensors = encoder_generator(valid_sentences,encoded_valid_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

Original:  pero es que tu eres la oveja negra y tu solito te lías
Token IDs: tensor([    4,  1355,  1058,  1038,  1294,  2149,  1030, 27836,  9715,  1042,
         1294,  1505,  1806,  1240,  2836,  1021,     5,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')


In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
valid_dataset = TensorDataset(valid_input_ids,valid_attention_masks,valid_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(valid_dataset)))

train data samples is 24856
valid data samples is 6216


In [ ]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(valid_dataset,
                              sampler=RandomSampler(valid_dataset),
                              batch_size=bs)

Load model BETO

In [ ]:
from transformers import AutoModel

beto = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
beto = beto.to(device)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Build CNN

In [ ]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [ ]:
# Hyperparameters

EMBEDDING_DIM = 768
N_FILTERS = 32
FILTER_SIZES = [1,2,3,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.2
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
cnn = cnn.to(device)

In [ ]:
cnn

CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
0.00002 == 2e-5

True

In [ ]:
# Optimizer and criterion

import torch.optim as optim

model_parameters = list(beto.parameters())+list(cnn.parameters())

optimizer = optim.Adam(model_parameters,lr=2e-5,eps=1e-8)
# optimizer = optim.Adam(model_parameters,lr=1e-3,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [ ]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    beto.train()
    cnn.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = beto(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval():
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    beto.eval()
    cnn.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = beto(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# # Set device and gpu

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

# beto.cuda()

# Training

In [ ]:
tname = "/content/beto_cnn_model_home"

In [ ]:
 epochs = 20
# epochs = 10
# epochs = 5

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(beto,f'{tname}_part1_train.pt')
        torch.save(cnn,f'{tname}_part2_train.pt')
        print("MODEL SAVED")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============\n')

100%|██████████| 49/49 [00:16<00:00,  2.93it/s]


              precision    recall  f1-score   support

           0       0.97      0.83      0.89      5538
           1       0.37      0.82      0.51       678

    accuracy                           0.83      6216
   macro avg       0.67      0.83      0.70      6216
weighted avg       0.91      0.83      0.85      6216

accuracy = 0.83
MODEL SAVED
Epoch: 01 | Epoch Time: 3m 28s
	Train Loss: 0.408 | Train acc: 81.98%
	 Val. Loss: 0.398 |  Val. acc: 82.56%
	 Val. F1: 70.09%
=============Epoch Ended==============



100%|██████████| 49/49 [00:17<00:00,  2.87it/s]


              precision    recall  f1-score   support

           0       0.94      0.85      0.89      5203
           1       0.49      0.73      0.58      1013

    accuracy                           0.83      6216
   macro avg       0.71      0.79      0.74      6216
weighted avg       0.87      0.83      0.84      6216

accuracy = 0.83
MODEL SAVED
Epoch: 02 | Epoch Time: 3m 34s
	Train Loss: 0.219 | Train acc: 91.35%
	 Val. Loss: 0.450 |  Val. acc: 83.06%
	 Val. F1: 73.93%
=============Epoch Ended==============



100%|██████████| 49/49 [00:17<00:00,  2.86it/s]


...

              precision    recall  f1-score   support

           0       0.95      0.85      0.90      5262
           1       0.47      0.76      0.58       954

    accuracy                           0.83      6216
   macro avg       0.71      0.80      0.74      6216
weighted avg       0.88      0.83      0.85      6216

accuracy = 0.83
Epoch: 18 | Epoch Time: 3m 36s
	Train Loss: 0.008 | Train acc: 99.69%
	 Val. Loss: 1.047 |  Val. acc: 83.37%
	 Val. F1: 73.94%
=============Epoch Ended==============



100%|██████████| 49/49 [00:17<00:00,  2.87it/s]


              precision    recall  f1-score   support

           0       0.96      0.84      0.90      5345
           1       0.45      0.79      0.58       871

    accuracy                           0.84      6216
   macro avg       0.71      0.82      0.74      6216
weighted avg       0.89      0.84      0.85      6216

accuracy = 0.84
Epoch: 19 | Epoch Time: 3m 35s
	Train Loss: 0.006 | Train acc: 99.80%
	 Val. Loss: 1.178 |  Val. acc: 83.67%
	 Val. F1: 73.75%
=============Epoch Ended==============



100%|██████████| 49/49 [00:17<00:00,  2.88it/s]

              precision    recall  f1-score   support

           0       0.94      0.86      0.90      5164
           1       0.52      0.75      0.61      1052

    accuracy                           0.84      6216
   macro avg       0.73      0.80      0.76      6216
weighted avg       0.87      0.84      0.85      6216

accuracy = 0.84
Epoch: 20 | Epoch Time: 3m 35s
	Train Loss: 0.006 | Train acc: 99.79%
	 Val. Loss: 1.120 |  Val. acc: 83.94%
	 Val. F1: 75.57%
=============Epoch Ended==============



In [ ]:
# Save BETO and CNN

torch.save(beto, tname + '_module_part1.pt')
torch.save(cnn,  tname + '_module_part2.pt')

In [ ]:
# # Save BETO and CNN
# tname = "/content/drive/MyDrive/TP1-AntiDiscriminator/weights/beto_cnn"
# torch.save(beto, tname + '_part1_train.pt')
# torch.save(cnn,  tname + '_part2_train.pt')

## EVALUATING

In [ ]:
# Load phobert and cnn

import torch
beto = torch.load(f'{tname}_part1_train.pt')
cnn = torch.load(f'{tname}_part2_train.pt')
beto.eval()
cnn.eval()


CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

Predict label from true label

In [ ]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    embedded = beto(b_input_ids,b_input_mask)[0]
    predictions = cnn(embedded)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

100%|██████████| 28/28 [00:09<00:00,  2.88it/s]


In [ ]:
# The final score in the test set (classification report)

#print(classification_report(all_pred_labels,all_true_labels, digits = 4))
print(classification_report(all_true_labels,all_pred_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9123    0.9113    0.9118      2932
           1     0.5925    0.5953    0.5939       635

    accuracy                         0.8551      3567
   macro avg     0.7524    0.7533    0.7528      3567
weighted avg     0.8553    0.8551    0.8552      3567



In [ ]:
# Confusion matrix in the test set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[2672,  260],
       [ 257,  378]])